## Chapter 2

In this Chapter, we refine our code through encapsulation - we create a class that can implement our first attempt at an options pricer.

In [11]:
import pandas as pd, numpy as np

In [19]:
from math import exp, sqrt, log
from random import random

In [13]:
class PayOff():
    def __init__(self,TheOptionsType_,Strike_):
        # The constructor
        self.__TheOptionsType = TheOptionsType_
        self.__Strike = Strike_
        
    def __call__(self,spot):
        # Overloading the ( ) operator
        if self.__TheOptionsType == 'call':
            return max(spot - self.__Strike,0)
        elif self.__TheOptionsType == 'put':
            return max(self.__Strike - spot,0)
        else:
            raise Exception('Unknown option type found.')


In [14]:
call = PayOff('call',80)
callPrice  = call(100)
print(f'Call price: {callPrice}')

Call price: 20


In [24]:
def SimpleMonteCarlo2(thePayOff, Expiry, Spot, Vol, r, NumberOfPaths):
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry+itoCorrection)
    thisSpot = 0
    runningSum = 0
    
    for i in range(NumberOfPaths):
        thisGaussian = np.random.normal()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayOff = thePayOff(thisSpot) 
        runningSum += thisPayOff
    
    mean = runningSum/NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean

In [39]:
df = pd.DataFrame(columns=['Call Price','Put Price','No. of Paths'])

We run the same example as in Chapter 1 using our updated programme.

In [40]:
CallPayOff = PayOff('call',40)
PutPayOff = PayOff('put',40)
for i in range(1,200,10):
    i = i*i
    callPrice = SimpleMonteCarlo2(CallPayOff,0.5,42,0.2,0.1,10000)
    putPrice = SimpleMonteCarlo2(PutPayOff,0.5,42,0.2,0.1,10000)
    df.loc[i] = pd.Series({'Call Price':format(callPrice,'.2f'), 'Put Price':format(putPrice, '.2f'), 'No. of Paths':format(i,'.0f')})

In [41]:
df

,Call Price,Put Price,No. of Paths
1,4.69,0.81,1
121,4.77,0.81,121
441,4.70,0.83,441
961,4.78,0.79,961
1681,4.80,0.82,1681
2601,4.76,0.82,2601
3721,4.82,0.81,3721
5041,4.76,0.82,5041
6561,4.78,0.77,6561
8281,4.77,0.86,8281
